# Тема 6.4. Логирование и анализ полетных данных

## Немного общей теории

### Определение

**Логирование** - процесс записи событий, происходящих в системе, программе или приложении, в специальные файлы-журналы (логи).

### Зачем

- Отладка
- Анализ производительности
- Расследование инцидентов

### Форматы хранения логов

#### CSV

Описание: Текстовый файл, где данные разделены запятыми (или точкой с запятой). Самый популярный формат для табличных данных

Плюсы:

- Человекочитаемость
- Легкая интеграция с библиотеками анализа
- Простота реализации

Минусы:

- Плохо подходит для иерархических данных (вложенных структур)
- Проблемы с разделителями (если в данных есть запятые)
- Занимает больше места, чем бинарный формат

#### JSON

Описание: Текстовый формат для хранения структурированных данных (словари, списки)

Плюсы:

- Поддержка сложной вложенности
- "Самодокументируемость" (поля имеют имена)

Минусы:

- Избыточность (названия полей повторяются в каждой записи, много скобок)
- Медленный парсинг по сравнению с CSV/Binary (как раз из-за избыточности)

#### Binary

Описание: Запись байтов напрямую в память файла

Примечание: рассматривать на семинаре его не будем, но важно знать, что именно с ним работает MAVLink

Плюсы:

- Максимальная компактность (экономия места в памяти дрона)
- Максимальная скорость записи (критично для микроконтроллеров и частот > 100Гц)

Минусы:

- Нельзя прочитать "глазами"
- Зависимость от архитектуры (например, от endianness — порядка записи байтов)
- Нужно знать точную структуру для расшифровки (что из данных было целым, что float'ом, что строчкой)

### Уровни логирования

![Log Levels](log_levels.png)

## Организация логирования в Python

### Библиотека `logging`

In [ ]:
import logging
import sys

# logging.Logger - начальник.
# logging.Handler - курьер.
# logging.Formatter - стилист.
# logging.Filter - фильтр.

# Создадим сам логгер
logger = logging.getLogger("DroneController")
logger.setLevel(logging.DEBUG)

# Добавим обработчик для вывода в консоль
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)

# Добавим обработчик для вывода в файл
file_handler = logging.FileHandler("flight_data.csv", mode="w")
file_handler.setLevel(logging.DEBUG)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

# Создадим форматтеры для красоты вывода
console_formatter = logging.Formatter(
    "%(asctime)s :: %(levelname)s :: %(message)s"
)
file_formatter = logging.Formatter("%(message)s")

console_handler.setFormatter(console_formatter)
file_handler.setFormatter(file_formatter)


# Базовый фильтр ничего не умеет, поэтому создадим класс-наследник, который будет оставлять только записи нужного нам уровня
class SingleLevelFilter(logging.Filter):
    def __init__(self, level):
        self.level = level

    def filter(self, record):
        return record.levelno == self.level


file_filter = SingleLevelFilter(logging.DEBUG)
file_handler.addFilter(file_filter)

# Файл предполагается csv - закинем в него шапку
logger.debug("timestamp,x,y,z,vx,vy,vz")
# Проверим также работоспособность вывода в консоль
logger.info("OK")

2025-12-10 16:48:47,413 :: INFO :: OK


Наконец создадим файл логов, запустив простенькую полетную миссию

In [2]:
import time

import numpy as np
from control import ROSInterface

drone = ROSInterface(logger=logger)
time.sleep(2)
drone.takeoff(5.0)
drone.fly_to_pose(np.array([20.0, 15.0, 10.0]))
drone.fly_to_pose(np.array([0.0, 0.0, 5.0]))
drone.shutdown()

2025-12-10 16:48:47,664 :: INFO :: ROS 2 node initialized.
2025-12-10 16:48:47,685 :: INFO :: ROS 2 spin thread started.
2025-12-10 16:48:49,692 :: INFO :: Takeoff to altitude 5.0
2025-12-10 16:48:59,160 :: INFO :: Flying to pose [20. 15. 10.]
2025-12-10 16:49:26,587 :: INFO :: Flying to pose [0. 0. 5.]
2025-12-10 16:49:54,052 :: INFO :: Shutting down Drone Controller...
2025-12-10 16:49:54,053 :: INFO :: Waiting for spin thread to finish...
2025-12-10 16:49:54,056 :: INFO :: ROS 2 spin thread stopped.
2025-12-10 16:49:54,056 :: INFO :: Destroying node drone_controller...
2025-12-10 16:49:54,058 :: INFO :: Node destroyed.
2025-12-10 16:49:54,058 :: INFO :: Shutting down rclpy...
2025-12-10 16:49:54,058 :: INFO :: rclpy shut down.
2025-12-10 16:49:54,058 :: INFO :: Drone Controller shutdown complete.
